In [13]:
import pandas as pd
import os

# Ruta de la carpeta donde están los archivos CSV
carpeta = "D:/HUGO/MAtriX/variables"

# Para leer todos los archivos csv dentro de esa carpeta
# Usamos os.path.join para construir rutas de forma segura en cualquier SO
archivos = [os.path.join(carpeta, f) for f in os.listdir(carpeta) if f.endswith(".csv")]

# Leer y combinar los archivos 1 por 1
data = None

for archivo in archivos:
    df = pd.read_csv(archivo)

    # El valor de la variable se llama Value, y está siempre en la ultima columna
    # Renombrar la última columna usando el prefijo antes del "–"
    # y eliminando espacios
    nombre_base = os.path.basename(archivo)
    # Extrae el prefijo antes del "–" o el primer espacio si "–" no existe
    if '–' in nombre_base:
        nuevo_nombre = nombre_base.split('–')[0].strip()
    else:
        nuevo_nombre = nombre_base.split(' ')[0].strip()

    # Eliminar la extensión .csv si aún está presente
    nuevo_nombre = nuevo_nombre.replace(".csv", "")
    # Borrar espacios adicionales
    nuevo_nombre = nuevo_nombre.replace(" ", "")

    if len(df.columns) > 0:
        # Renombra la última columna
        df.rename(columns={df.columns[-1]: nuevo_nombre}, inplace=True)
        print(f"{nuevo_nombre} Número de datos: {len(df)}")

    if data is None:
        data = df  # Si es el primer archivo, se asigna directamente
    else:
        # Columnas comunes para la unión
        columnas_comunes = list(set(data.columns) & set(df.columns))
        # Une por esas columnas (full_join en pandas es merge con how='outer')
        data = pd.merge(data, df, on=columnas_comunes, how='outer')

# Ahora 'data' contiene el df combinado con las variables predictoras y las coordenadas
# print(data.head(5)) # Descomentar para ver las primeras filas

A Número de datos: 67212
AC Número de datos: 67212
B Número de datos: 67212
CIB Número de datos: 67212
CICCB Número de datos: 67212
CUS Número de datos: 67212
DBB Número de datos: 67212
DOB Número de datos: 67212
OC Número de datos: 67212
P Número de datos: 67212


In [14]:
# Ver si hay valores faltantes en este punto
print(data.isna().sum())

grid_cod    0
POINT_X     0
POINT_Y     0
A           0
AC          0
B           0
CIB         0
CICCB       0
CUS         0
DBB         0
DOB         0
OC          0
P           0
dtype: int64


Una vez se tienen las variables predictoras, se añade también la variable respuesta PLA (profundidad de la lámina de agua).

In [15]:
# Cargar el nuevo CSV PLA con separador ";" y decimales con ","
archivo_nuevo = "D:/HUGO/MAtriX/variables/PLA/CSV_PLA_19_05_25.csv"

# Para leer un CSV con separador ';' y decimales ',' en pandas, usamos 'sep' y 'decimal'
df_nuevo = pd.read_csv(archivo_nuevo, sep=',', decimal='.')
print(df_nuevo.head(4))

   grid_cod        POINT_X       POINT_Y  Value
0      6909  722404.251470  4.365852e+06    0.0
1      7246  722396.507981  4.365860e+06    0.0
2      7247  722404.245816  4.365860e+06    0.0
3      7248  722411.983652  4.365860e+06    0.0


In [16]:
# Seleccionar las coordenadas y la columna Value, renombrándola como PLA
# Usamos .loc para seleccionar columnas y .rename para renombrar
df_nuevo = df_nuevo.loc[:, ['POINT_X', 'POINT_Y', 'Value']].rename(columns={'Value': 'PLA'})

In [17]:
# Convertir coordenadas a un mismo tipo (numérico)
# Usamos .astype(float) para asegurar que sean números de punto flotante
df_nuevo['POINT_X'] = df_nuevo['POINT_X'].astype(float)
df_nuevo['POINT_Y'] = df_nuevo['POINT_Y'].astype(float)

# Asumiendo que 'data' ya es un DataFrame de pandas de los pasos anteriores
# Convertir coordenadas de 'data' a numérico también (si no lo están ya)
data['POINT_X'] = data['POINT_X'].astype(float)
data['POINT_Y'] = data['POINT_Y'].astype(float)

# Realizar un left_join para encontrar coordenadas coincidentes
# Seleccionamos las columnas de interés y luego usamos pd.merge con how='inner'
# Un inner join es lo que necesitas para encontrar solo las filas que coinciden en ambos DataFrames.
# Si solo quieres las coordenadas de 'data' que tienen coincidencia en 'df_nuevo',
# un 'left' join sería adecuado, pero para 'coordenadas_comunes' que son las que *ambos* tienen, 'inner' es el camino.
coordenadas_comunes = pd.merge(
    data[['POINT_X', 'POINT_Y']],
    df_nuevo[['POINT_X', 'POINT_Y']],
    on=['POINT_X', 'POINT_Y'],
    how='inner' # Para encontrar solo las coordenadas coincidentes
)

# Imprimir el número de coordenadas coincidentes
print(f"Número de coordenadas coincidentes: {len(coordenadas_comunes)}")

Número de coordenadas coincidentes: 67212


Vemos que las coordenadas coinciden con el número de datos que obtuvimos al principio.

In [19]:
# Unir el nuevo CSV (df_nuevo) con data
# Usamos pd.merge con how='inner' para el inner_join
data = pd.merge(data, df_nuevo, on=['POINT_X', 'POINT_Y'], how='inner')

# Reordenar las columnas para que PLA sea la tercera
# Primero, definimos el orden deseado para las columnas principales
columnas_principales = ["POINT_X", "POINT_Y", "PLA"]

# Luego, obtenemos el resto de las columnas que no sean las principales,
# manteniendo su orden original y asegurándonos de que 'PLA' esté ahora en 'data'
otras_columnas = [col for col in data.columns if col not in columnas_principales]

# Creamos la lista final de columnas en el orden deseado
columnas_ordenadas = columnas_principales + otras_columnas

# Reordenamos el DataFrame
data = data[columnas_ordenadas]

# Mostrar las primeras filas del dataset combinado
print("Primeras filas del DataFrame combinado:")
print(data.head(3))

# Ver si hay valores faltantes en este punto
print("\nConteo de valores faltantes (NaN) en el DataFrame combinado:")
print(data.isna().sum())

print("\nConteo total de True/False para valores faltantes en todo el DataFrame:")
total_faltantes = data.isna().sum().sum()
total_no_faltantes = data.size - total_faltantes
print(f"Total de valores faltantes (True): {total_faltantes}")
print(f"Total de valores no faltantes (False): {total_no_faltantes}")


# Guardar el resultado en un nuevo CSV
# write_csv en R es to_csv en pandas
nombre_archivo_salida = "datos_combinados.csv"
data.to_csv(nombre_archivo_salida, index=False) # index=False evita guardar el índice de pandas como una columna

print(f"\nDataFrame combinado guardado exitosamente como '{nombre_archivo_salida}'")

Primeras filas del DataFrame combinado:
         POINT_X       POINT_Y  PLA  grid_cod          A  AC  B          CIB  \
0  722404.251470  4.365852e+06  0.0      6909  17.955999  11  0 -9999.000000   
1  722396.507981  4.365860e+06  0.0      7246  18.179501   4  0     0.271946   
2  722404.245816  4.365860e+06  0.0      7247  18.274900  11  0     0.269777   

      CICCB  CUS         DBB          DOB  OC         P  
0  0.448429    1  848.570984     0.000000   4  4.771400  
1  0.450929    5  855.966980     0.000000   4  1.352900  
2  0.449347    1  847.986023  1110.060059   4  0.436232  

Conteo de valores faltantes (NaN) en el DataFrame combinado:
POINT_X     0
POINT_Y     0
PLA         0
grid_cod    0
A           0
AC          0
B           0
CIB         0
CICCB       0
CUS         0
DBB         0
DOB         0
OC          0
P           0
dtype: int64

Conteo total de True/False para valores faltantes en todo el DataFrame:
Total de valores faltantes (True): 0
Total de valores no faltan